### Genetic Algorithm Exercise


In [1]:
import random

# Genetic Algorithm Parameters
population_size = 20
chromosome_length = 5  # Binary representation length for numbers 0 to 31 (5 bits)
mutation_rate = 0.1
stop_limit = 20  # Number of generations with no improvement allowed

#### Defining All Functions That will be required in main algorithm

In [2]:
# Fitness function: x^2
def fitness_function(x):
    return x ** 2

# Generate initial population
def generate_population(size, chromosome_length):
    population = []
    for _ in range(size):
        individual = [random.choice([0, 1]) for _ in range(chromosome_length)]
        population.append(individual)
    return population

# Decode binary gene to integer
def decode_gene(gene):
    value = 0
    for bit in gene:
        value = (value << 1) | bit  # Convert binary array to integer
    return value

# Evaluate fitness for all individuals
def evaluate_fitness(population):
    fitness_scores = []
    for individual in population:
        x = decode_gene(individual)
        fitness_scores.append(fitness_function(x))
    return fitness_scores

# Selection using expected count formula
def select_population(population, fitness_scores):
    # Calculate average fitness
    avg_fitness = sum(fitness_scores) / len(fitness_scores)
    
    # Calculate expected counts based on the formula
    expected_counts = []
    for fitness in fitness_scores:
        expected_count = round(fitness / avg_fitness)
        expected_counts.append(expected_count)
    
    # Adjust the list to ensure the total matches population size
    total_count = sum(expected_counts)
    while total_count > population_size:
        max_index = expected_counts.index(max(expected_counts))
        expected_counts[max_index] -= 1
        total_count -= 1
    while total_count < population_size:
        max_index = fitness_scores.index(max(fitness_scores))
        expected_counts[max_index] += 1
        total_count += 1
    
    # Generate the selected population
    selected = []
    for individual, count in zip(population, expected_counts):
        for _ in range(count):
            selected.append(individual)
    
    random.shuffle(selected)
    return selected

# Perform single-point crossover
def crossover(parent1, parent2):
    point = random.randint(1, chromosome_length - 1)  # Choose a crossover point
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

In [3]:
# Perform mutation
def mutate(individual):
    mutated = []
    for bit in individual:
        if random.random() < mutation_rate:
            mutated.append(1 - bit)  # Flip the bit
        else:
            mutated.append(bit)
    return mutated

#### Main Genetic Algorithm

In [4]:
def genetic_algorithm():
    # Step 1: Initialize population
    population = generate_population(population_size, chromosome_length)
    best_fitness = 0
    stagnation_counter = 0
    
    generation = 0
    best_individual = None
    while stagnation_counter < stop_limit:
        generation += 1
        
        # Step 2: Evaluate fitness
        fitness_scores = evaluate_fitness(population)
        max_fitness = max(fitness_scores)
        best_individual = population[fitness_scores.index(max_fitness)]
        
        # Check for improvement in fitness
        if max_fitness > best_fitness:
            best_fitness = max_fitness
            stagnation_counter = 0  # Reset counter if improvement occurs
        else:
            stagnation_counter += 1  # Increment stagnation counter
        
        # Step 3: Selection
        selected_population = select_population(population, fitness_scores)
        
        # Step 4: Crossover
        next_population = []
        for i in range(0, population_size, 2):
            parent1 = selected_population[i]
            parent2 = selected_population[i + 1]
            child1, child2 = crossover(parent1, parent2)
            next_population.append(child1)
            next_population.append(child2)
        
        # Step 5: Mutation
        population = [mutate(individual) for individual in next_population]

    
    best_sol=decode_gene(best_individual)
    # Return the best individual, its fitness, and the total number of generations
    return best_sol, best_fitness, generation

#### Run the Genetic Algorithm

In [5]:
best_sol, best_fitness, generations = genetic_algorithm()
print(f"Best solution: {best_sol}")
print(f"Fitness of best solution: {best_fitness}")
print(f"Total generations: {generations}")

Best solution: 31
Fitness of best solution: 961
Total generations: 21
